In [1]:
%matplotlib qt

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as mcolors
import numpy as np
import os
import random
from pathlib import Path

# ==========================================
# 1. CONFIGURATION & PARAMETERS
# ==========================================

USE_DUMMY_DATA = True 

# Base directories (relative paths)
BASE_DIR = Path.cwd()              # Jupyter notebook directory
PROJECT_ROOT = BASE_DIR.parent     # Folder containing /data/

base_gen = PROJECT_ROOT / "data" / "generated"
base_output = PROJECT_ROOT / "data" / "generated"

# File paths
loc_types_path = base_gen / "location_types_generated.csv"
full_locs_output = base_output / "locations.csv"
dummy_output = base_output / "locations_dummy.csv"

# Layout Constraints (in Millimeters)
MAX_RACK_HEIGHT = 2200    
ROW_LENGTH_LIMIT = 2500  
AISLE_WIDTH = 1200        
RACK_SPACING = 50        

# ==========================================
# 2. DATA LOADING
# ==========================================

def get_location_types():
    if USE_DUMMY_DATA:
        data = {
            'LOC_CODE': ['A1', 'A2', 'A3'],
            'WID_MM':   [160.0, 240.0, 410.0],
            'HT_MM':    [130.0, 220.0, 260.0],
            'DEP_MM':   [600.0, 790.0, 850.0],
            'NUM_LOCS': [63, 204, 90]
        }
        return pd.DataFrame(data)
    else:
        if not os.path.exists(loc_types_path):
            raise FileNotFoundError(f"File not found: {loc_types_path}")
        return pd.read_csv(loc_types_path)

df_types = get_location_types()
print(f"Loaded {len(df_types)} location types.")

# ==========================================
# 3. HYBRID LAYOUT ENGINE
# ==========================================

def generate_hybrid_layout(df_types):
    generated_locations = []
    
    # Global Coordinates
    current_x = 0
    current_y = 0      
    current_z = 0
    
    # Semantic Counters
    row_counter = 1
    bay_counter = 1
    level_counter = 1
    
    # Trackers
    current_row_max_depth = 0
    prev_type = None
    prev_width = 0
    
    for index, row in df_types.iterrows():
        l_code = row['LOC_CODE']
        width = row['WID_MM']
        height = row['HT_MM']
        depth = row['DEP_MM']
        total_count = int(row['NUM_LOCS'])
        
        print(f"Adding {total_count} units of {l_code}...")
        
        for i in range(total_count):
            
            # --- LOGIC 1: Type Transition (Force New Bay) ---
            if prev_type is not None and l_code != prev_type:
                if current_z > 0: 
                    current_z = 0
                    current_x += (prev_width + RACK_SPACING)
                    bay_counter += 1
                    level_counter = 1

            if depth > current_row_max_depth:
                current_row_max_depth = depth
            
            # --- LOGIC 2: Vertical Stacking (New Bay if Full) ---
            if (current_z + height) > MAX_RACK_HEIGHT:
                current_z = 0
                current_x += (width + RACK_SPACING)
                bay_counter += 1
                level_counter = 1 
            
            # --- LOGIC 3: Row Length Limit (New Row Trigger) ---
            if current_x > ROW_LENGTH_LIMIT:
                current_x = 0
                current_z = 0
                
                current_y += current_row_max_depth
                
                if row_counter % 2 != 0:
                    current_y += AISLE_WIDTH
                else:
                    current_y += 20 
                
                current_row_max_depth = depth 
                
                row_counter += 1
                bay_counter = 1
                level_counter = 1

            # --- LOGIC 4: Create IDs and Columns ---
            
            # A. The Unique Sequential ID (Restored)
            loc_id = f"{l_code}-{i+1:05d}"
            
            # B. The Human Readable Address (Optional string)
            address_str = f"R{row_counter:02d}-B{bay_counter:03d}-L{level_counter:02d}"
            
            loc_data = {
                'loc_inst_code': loc_id,      # Unique Key: A1-00001
                'loc_type': l_code,
                'x': current_x,
                'y': current_y,
                'z': current_z,
                'width': width,
                'depth': depth,
                'height': height,
                # --- NEW COLUMNS ADDED HERE ---
                'row_num': row_counter,
                'bay_num': bay_counter,
                'level_num': level_counter,
                'address_label': address_str  # Helper for UI
            }
            generated_locations.append(loc_data)
            
            current_z += height
            level_counter += 1
            prev_type = l_code
            prev_width = width

    return pd.DataFrame(generated_locations)

# Run Generation
df_layout = generate_hybrid_layout(df_types)

# Save
if not USE_DUMMY_DATA:
    os.makedirs(os.path.dirname(full_locs_output), exist_ok=True)
    df_layout.to_csv(full_locs_output, index=False)
else:
    os.makedirs(os.path.dirname(dummy_output), exist_ok=True)
    df_layout.to_csv(dummy_output, index=False)

# ==========================================
# 4. VISUALIZATION (TOP VIEW)
# ==========================================

def plot_warehouse_map(df_layout):
    fig, ax = plt.subplots()
    ax.set_facecolor('white') 
    
    unique_types = df_layout['loc_type'].unique()
    colors = plt.cm.get_cmap('tab10', len(unique_types))
    type_color_map = {ctype: colors(i) for i, ctype in enumerate(unique_types)}
    
    df_footprint = df_layout.drop_duplicates(subset=['x', 'y', 'loc_type'])
    
    print(f"Drawing Top View ({len(df_footprint)} stacks)...")

    for _, row in df_footprint.iterrows():
        rect = patches.Rectangle(
            (row['x'], row['y']), 
            row['width'], 
            row['depth'], 
            linewidth=0.5, 
            edgecolor='black', 
            facecolor=type_color_map[row['loc_type']],
            alpha=0.8
        )
        ax.add_patch(rect)

    ax.autoscale()
    ax.set_aspect('equal')
    ax.invert_yaxis()
    
    ax.set_title("Warehouse Top View", fontsize=16)
    ax.set_xlabel("Rack Length (mm)")
    ax.set_ylabel("Warehouse Depth (mm)")
    
    handles = [patches.Patch(color=type_color_map[t], label=t) for t in unique_types]
    ax.legend(handles=handles, title="Location Types", loc='upper right')

    manager = plt.get_current_fig_manager()
    try: manager.window.showMaximized()
    except: pass

    plt.show()

# ==========================================
# 5. FRONT VIEW VISUALIZATION
# ==========================================

def get_utilization_data(loc_id):
    return random.random()

def get_traffic_light_color(percentage):
    if percentage < 0.50: return '#2ecc71'
    elif percentage < 0.85: return '#f1c40f'
    else: return '#e74c3c'

def plot_front_view(df_layout, start_loc_code, end_loc_code):
    try:
        # Sort by the unique code to find range
        df_sorted = df_layout.sort_values('loc_inst_code').reset_index(drop=True)
        
        indices_start = df_sorted[df_sorted['loc_inst_code'] == start_loc_code].index
        indices_end = df_sorted[df_sorted['loc_inst_code'] == end_loc_code].index
        
        if len(indices_start) == 0:
            print(f"ERROR: Start Location '{start_loc_code}' not found.")
            return
        if len(indices_end) == 0:
            print(f"ERROR: End Location '{end_loc_code}' not found.")
            return
            
        idx_start = indices_start[0]
        idx_end = indices_end[0]
        
        if idx_start > idx_end:
            idx_start, idx_end = idx_end, idx_start
            
        subset_range = df_sorted.iloc[idx_start : idx_end+1]
        
    except Exception as e:
        print(f"An error occurred during selection: {e}")
        return

    # Expand to full vertical stacks
    target_positions = subset_range[['x', 'y']].drop_duplicates()
    full_stacks = pd.merge(df_layout, target_positions, on=['x', 'y'], how='inner')
    
    # Sort by Row/Bay/Level for clean plotting order
    full_stacks = full_stacks.sort_values(by=['row_num', 'bay_num', 'level_num'])

    fig, ax = plt.subplots(figsize=(15, 8))
    ax.set_facecolor('white')
    
    unique_stacks = {} 
    for _, row in full_stacks.iterrows():
        coord_key = (row['x'], row['y'])
        if coord_key not in unique_stacks:
            unique_stacks[coord_key] = []
        unique_stacks[coord_key].append(row)

    cursor_x = 0 
    GAP_BETWEEN_RACKS = 50 
    
    print(f"Plotting Front View for range {start_loc_code} to {end_loc_code}...")

    max_plot_height = 0

    for coord, locs_in_stack in unique_stacks.items():
        stack_width = locs_in_stack[0]['width']
        
        for loc in locs_in_stack:
            h = loc['height']
            z = loc['z']
            lid = loc['loc_inst_code']
            
            # Visualization Logic
            util_pct = get_utilization_data(lid)
            fill_height = h * util_pct
            color = get_traffic_light_color(util_pct)
            
            # Border
            border_rect = patches.Rectangle(
                (cursor_x, z), stack_width, h,
                linewidth=1, edgecolor='black', facecolor='none', zorder=2
            )
            ax.add_patch(border_rect)
            
            # Fill
            if util_pct > 0:
                fill_rect = patches.Rectangle(
                    (cursor_x, z), stack_width, fill_height,
                    linewidth=0, facecolor=color, alpha=0.8, zorder=1
                )
                ax.add_patch(fill_rect)
                
            # LABEL: Using the new semantic 'level_num' column
            if h > 50 and stack_width > 50:
                level_label = f"L{loc['level_num']}"
                ax.text(cursor_x + stack_width/2, z + h/2, level_label, 
                        ha='center', va='center', fontsize=6, color='#333')

        top_of_stack = locs_in_stack[-1]['z'] + locs_in_stack[-1]['height']
        if top_of_stack > max_plot_height:
            max_plot_height = top_of_stack

        cursor_x += (stack_width + GAP_BETWEEN_RACKS)

    ax.set_xlim(-100, cursor_x + 100)
    ax.set_ylim(0, max_plot_height + 200)
    ax.set_aspect('equal')
    
    ax.set_title(f"Rack Elevation: {start_loc_code} -> {end_loc_code}", fontsize=14)
    ax.set_xlabel("Rack Sequence (Bay Direction)")
    ax.set_ylabel("Elevation (mm)")

    manager = plt.get_current_fig_manager()
    try: manager.window.showMaximized()
    except: pass
    
    plt.show()

# ==========================================
# EXAMPLE USAGE
# ==========================================

# 1. Plot Top View
plot_warehouse_map(df_layout)

# 2. Print columns to verify structure
print("\n--- SAMPLE DATA (Columns) ---")
print(df_layout[['loc_inst_code', 'row_num', 'bay_num', 'level_num', 'address_label']].head(5))

# 3. Plot Front View using SEQUENTIAL IDs
start_node = 'A1-00001'
end_node   = 'A1-00063'  # This should capture the full first block of A1

plot_front_view(df_layout, start_node, end_node)

Loaded 3 location types.
Adding 63 units of A1...
Adding 204 units of A2...
Adding 90 units of A3...


C:\Users\arthu\AppData\Local\Temp\ipykernel_30528\1513460122.py:179: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = plt.cm.get_cmap('tab10', len(unique_types))


Drawing Top View (37 stacks)...

--- SAMPLE DATA (Columns) ---
  loc_inst_code  row_num  bay_num  level_num address_label
0      A1-00001        1        1          1  R01-B001-L01
1      A1-00002        1        1          2  R01-B001-L02
2      A1-00003        1        1          3  R01-B001-L03
3      A1-00004        1        1          4  R01-B001-L04
4      A1-00005        1        1          5  R01-B001-L05
Plotting Front View for range A1-00001 to A1-00063...
